# Analyzing MC-results for clearance experiments (and last-mile computations)

Here, we'll take the raw outputs from the experiments and post-process them into the final Monte-Carlo results we'll be looking for.
An entry of one MC-iteration is the relevant experiment params (clearance, n_samples in the PRM, dimension), and the `threshold' number of neighbors/radius to find a feasible path (fewer neighbors/shorter results in no path solve). 

This allows is to use the threshold to compute the probability of a wide range of radii/neighbors (have a discrete grid of radii/neighbors, increment the ones above the threshold).

In [1]:
%cd ~/Research/nonasymptotic-mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import itertools as it
import json
import glob
import os

/home/seiji/Research/nonasymptotic-mp


In [2]:
# load in the experiment and set plotting params
exp_name = 'narrow-full-run9_20240522-081521'
exp_path = os.path.join('exps', 'results', exp_name)

n_radii = 100

Loading in the experiment log:

In [3]:
out_paths = glob.glob(os.path.join(exp_path, 'out*.csv'))
out_df = pd.concat(map(lambda x: pd.read_csv(x, index_col=0), out_paths))

exp_config = json.load(open(os.path.join(exp_path, 'config.json'), 'r'))
max_rad = exp_config['radius_prm_max_radius']
max_knn = exp_config['knn_prm_max_neighbors']

rads = np.linspace(out_df['conn_lb'].min(), max_rad, num=n_radii)

Doing the MC-computation for radius/knns

In [4]:
# first prep the corresponding counters for all the mc trials. 
# we'll do a relatively naive dictionary-type thing, and convert floats to strings to use as keys
# create iterators for each experiment type and stack them together
radius_trials_iter = it.product(
    ['radius'],  # need this for proper indexing into the pandas array
    exp_config['deltas'],
    exp_config['dims_to_test'],
    exp_config['radius_sample_schedule'],
)

knn_trials_iter = it.product(
    ['knn'],  # need this for proper indexing into the pandas array
    exp_config['deltas'],
    exp_config['dims_to_test'],
    exp_config['knn_sample_schedule'],
)

all_trials_iter = it.chain(radius_trials_iter, knn_trials_iter)
estimate_dict = {}

for prm_type, clearance, dim, n_samples in all_trials_iter:
    _clearance = str(clearance)

    # filter for the specific trial type
    mc_trial_rows = out_df[
        (out_df['prm_type'] == prm_type) &
        (out_df['dim'] == dim) &
        np.isclose(out_df['delta_clearance'], clearance) &
        (out_df['n_samples'] == n_samples)
        ]

    if prm_type == 'knn':
        conn_thresh_arr = np.tile(
            mc_trial_rows['conn_ub'].to_numpy().reshape(-1, 1),
            (1, max_knn)
        )
        conn_cmp_arr = np.tile(np.arange(max_knn), (conn_thresh_arr.shape[0], 1))
    else:
        conn_thresh_arr = np.tile(
            mc_trial_rows['conn_ub'].to_numpy().reshape(-1, 1),
            (1, n_radii)
        )
        conn_cmp_arr = np.tile(rads, (conn_thresh_arr.shape[0], 1))

    above_threshold = conn_cmp_arr >= conn_thresh_arr
    estimates = np.sum(above_threshold, axis=0) / len(mc_trial_rows)
    estimate_dict[(prm_type, _clearance, dim, n_samples)] = estimates

Plotting results:
This isn't the final version of the plot (since we need to tabulate the data anyway because there is too much otherwise),
but these line plots is a good way to understand how the empirical probability changes as the
radius changes.

We'll also compare against the theoretical bound too. We'll work with the feasibility version.
For passage of clearance $\delta$, then we'll require a $\delta/4$-net and a connection radius
of $2\delta$. 

If the largest radius turns out to be smaller than $2\delta$, then we'll just compute
the bound for the corresponding smaller net (as dictated by the radius). We'll need to expect some
looseness for that scenario. NOTE: We tried this, and it turns out the numerical bound computation is vacuous in this scenario. Even just using the largest net rad as dictated by the environment is off by a full order of magnitude.

In [9]:
from nonasymptotic.bound import compute_sauer_shelah_bound_log2, compute_vol_unit_sphere
from nonasymptotic.envs import NarrowPassage

# recompile the results into a dataframe and then use seaborn to plot.
# the most efficient way to do this is to set up each array separately and then concatenate
plot_dfs = []
for prm_type, _clearance, dim, n_samples in estimate_dict.keys():
    clearance = float(_clearance)
    estimates = estimate_dict[(prm_type, _clearance, dim, n_samples)]  
   
    # if the estimate is nan, then it hasn't been processed yet by the experiments. so we skip.
    if np.isnan(estimates[-1]):
        continue
      
    # put estimates format amenable for dataframe
    dict_for_df = {
        "p": estimates,
        "conn": np.arange(max_knn) if prm_type == 'knn' else rads,
        "p_type": ["estimated"] * max_knn if prm_type == 'knn' else ["estimated"] * n_radii,
    }
    
    if prm_type == 'radius':
        # compute corresponding theoretical quantities (using smallest Sauer-Shelah expression)
        vol_env = NarrowPassage(dim, clearance, seed=1999).vol
        net_rads = np.array([clearance / 4] * n_radii) #np.minimum(rads / 8, clearance / 4) # clip for net radius corresponding to clearance of the passage
        rhos = (compute_vol_unit_sphere(dim) / vol_env) * (net_rads ** dim)
        log2_ss_probs = np.array([compute_sauer_shelah_bound_log2(n_samples, _rho, dim + 1) for _rho in rhos])
        ss_probs = 2 ** log2_ss_probs
        dict_for_df['p'] = np.concatenate([dict_for_df['p'], ss_probs])
        dict_for_df['conn'] = np.concatenate([dict_for_df['conn'], rads])
        dict_for_df['p_type'] += ["theory"] * n_radii
    
    # construct dataframe
    _df = pd.DataFrame(dict_for_df)
    _df['prm_type'] = prm_type
    _df['clearance'] = _clearance
    _df['dim'] = dim
    _df['n_samples'] = n_samples
    plot_dfs.append(_df)
plot_df = pd.concat(plot_dfs)    

# in this scenario, it makes sense to plot knn and radius prms separately
radius_plot_df = plot_df[plot_df['prm_type'] == 'radius']
knn_plot_df = plot_df[plot_df['prm_type'] == 'knn']

plt.figure()
sns.set_theme(style="whitegrid")
sns.relplot(data=radius_plot_df, x='conn', y='p', row='dim', col='clearance', hue='n_samples',  style='p_type',
            kind='line', palette=sns.color_palette("pastel"))
plt.title('Radius MC Experiments')
plt.show()

plt.figure()
sns.set_theme(style="whitegrid")
sns.relplot(data=knn_plot_df, x='conn', y='p', row='dim', col='clearance', hue='n_samples',  style='p_type',
            kind='line', palette=sns.color_palette("pastel"))
plt.title('KNN MC Experiments')
plt.show()

KeyboardInterrupt: 